In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, models, optimizers
from tensorflow.keras.layers import TimeDistributed, Conv2D, Flatten, Dense, LSTM, ConvLSTM2D, MaxPool2D, Dropout
import matplotlib.pyplot as plt

import numpy as np 
import pandas as pd

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
tf.compat.v1.enable_eager_execution()

In [4]:
inputs = np.load("inputs_train.npy")

In [5]:
inputs_train = np.load("inputs_train.npy")
inputs_test = np.load("inputs_test.npy")

In [6]:
inputs_train = np.transpose(inputs_train, (0, 2,3,1))
inputs_test = np.transpose(inputs_test, (0, 2,3,1))

In [7]:
train = pd.read_csv('train_timeframes.csv').iloc[59:]
test = pd.read_csv('test_timeframes.csv').iloc[59:]

In [8]:
df = [train, test]

In [9]:
img = inputs_train[0,:,:,:]

In [10]:
inputs_train.shape, train.shape

((73695, 60, 4, 5), (73695, 23))

In [11]:
inputs_test.shape, test.shape

((24365, 60, 4, 5), (24365, 23))

In [98]:
idx = 0
batch_size = 32
stop_limit = 0.004
price_limit = 0.004
margin_size = 50
time_limit = 120

In [12]:
from keras.layers import ConvLSTM2D, Bidirectional, BatchNormalization, Conv2D, Cropping2D, ZeroPadding2D
from keras.models import Sequential
sim_model = Sequential()
sim_model.add(BatchNormalization(input_shape = (None, None, None, 1)))
sim_model.add(Conv2D(8, 
                     kernel_size = (1, 5), 
                     padding = 'same',
                     activation = 'relu'))
sim_model.add(Conv2D(8, 
                     kernel_size = (3, 3), 
                     padding = 'same',
                     activation = 'relu'))
sim_model.add(BatchNormalization())
sim_model.add(Bidirectional(ConvLSTM2D(16, 
                                       kernel_size = (3, 3),
                                       padding = 'same',
                                       return_sequences = True)))
sim_model.add(Bidirectional(ConvLSTM2D(32, 
                                       kernel_size = (3, 3),
                                       padding = 'same',
                                       return_sequences = True)))
sim_model.add(Conv2D(8, 
                     kernel_size = (1, 3), 
                     padding = 'same',
                     activation = 'relu'))
sim_model.add(Conv2D(1, 
                     kernel_size = (1,1), 
                     activation = 'sigmoid'))
sim_model.add(Cropping2D((1, 2))) # avoid skewing boundaries
sim_model.add(ZeroPadding2D((1, 2)))
sim_model.summary()

ValueError: Input 0 of layer cropping2d is incompatible with the layer: expected ndim=4, found ndim=5. Full shape received: (None, None, None, None, 1)

In [ ]:
model = models.Sequential()
model.add(tf.keras.Input(shape=(None,) + img.shape))
model.add(ConvLSTM2D(64, kernel_size=(5, 1), strides=(1, 1), activation='relu', padding='same', return_sequences=True))
# model.add(ConvLSTM2D(64, kernel_size=(15, 1), strides=(1, 1), activation='relu', padding='same'))
# model.add(ConvLSTM2D(64, kernel_size=(30, 1), strides=(1, 1), activation='relu', padding='same'))
model.add(Flatten())
model.add(Dense(3, activation="sigmoid"))

In [ ]:
model = models.Sequential()
model.add(TimeDistributed(Conv2D(64, kernel_size=(5, 1), strides=(1, 1), activation='relu', padding='same', input_shape=(None,) + img.shape)))
model.add(TimeDistributed(Conv2D(64, kernel_size=(15, 1), strides=(1, 1), activation='relu', padding='same')))
model.add(TimeDistributed(Conv2D(64, kernel_size=(30, 1), strides=(1, 1), activation='relu', padding='same')))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64))
model.add(LSTM(32))
model.add(Dense(3, activation="sigmoid"))

In [ ]:
model = models.Sequential()
model.add(TimeDistributed(Conv2D(64, kernel_size=(5, 1), strides=(1, 1), activation='relu', padding='same', input_shape=(None,) + img.shape)))
model.add(TimeDistributed(Conv2D(64, kernel_size=(15, 1), strides=(1, 1), activation='relu', padding='same')))
model.add(TimeDistributed(Conv2D(64, kernel_size=(30, 1), strides=(1, 1), activation='relu', padding='same')))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64))
model.add(LSTM(32))
model.add(Dense(3, activation="sigmoid"))

In [13]:
model = models.Sequential()
model.add(Conv2D(64, kernel_size=(5, 1), strides=(1, 1), activation='relu', padding='same', input_shape=img.shape))
model.add(Conv2D(64, kernel_size=(15, 1), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(64, kernel_size=(30, 1), strides=(1, 1), activation='relu', padding='same'))
model.add(Flatten())
model.add(Dense(64))
model.add(Dense(3, activation="sigmoid"))

In [104]:
model = models.Sequential()

model.add(Conv2D(32, (5, 1),padding ='Same', activation='relu', input_shape = img.shape))
model.add(Conv2D(32, (5, 1),padding = 'Same', activation ='relu'))
model.add(MaxPool2D((2, 1)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3, 1),padding = 'Same', activation ='relu'))
model.add(Conv2D(64, (3, 1),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2, 1), strides=(2, 1)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(3, activation = "sigmoid"))

In [105]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_56 (Conv2D)           (None, 60, 4, 32)         832       
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 60, 4, 32)         5152      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 30, 4, 32)         0         
_________________________________________________________________
dropout_42 (Dropout)         (None, 30, 4, 32)         0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 30, 4, 64)         6208      
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 30, 4, 64)         12352     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 15, 4, 64)       

In [106]:
def custom_metric_wrapper(idx, stop_limit, price_limit, margin_size, time_limit):    
    def custom_metric(y_true, y_pred):
        margin = ((y_pred[:,:1] - 0.5) * 2)
        margin *= margin_size
        direction = tf.sign(margin)
        stop_loss = y_pred[:,1:2] * direction * -1 * stop_limit
        price_target = y_pred[:,2:3] * direction * 1 * price_limit
        
        idx = y_true[0].numpy()[1]
        df_name = int(y_true[0].numpy()[0])
        batch_size = y_pred.shape[0]
        closes = []
        spread = []
        for i in range(batch_size):
            closes.append(df[df_name].iloc[int(idx+i):int(idx+i+time_limit)]['close_1min'].tolist())
            spread.append(df[df_name].iloc[int(idx+i):int(idx+i+time_limit)]['spread'].tolist())
        closes = tf.convert_to_tensor(closes)
        spread = tf.convert_to_tensor(spread)
        spread *= direction
        
        close1 = closes[:,:1]
        close2 = tf.zeros_like(closes[:,:1])
        for i in range(1, closes.shape[1]):
            diff = (closes[:,i:i+1] - closes[:,:1]) * direction
            cond = tf.logical_or(tf.less(diff, stop_loss), tf.greater(diff, price_target))
            close2 = tf.where(tf.logical_and(tf.equal(close2, 0), cond), closes[:,i:i+1], close2)
            
        close2 = tf.where(tf.equal(close2, 0), closes[:,-1:], close2)

        diff = close2 - close1
        profit = 100 * (margin * (diff - spread)) /  close1
        return profit
    return custom_metric

def custom_loss_wrapper(idx, stop_limit, price_limit, margin_size, time_limit):    
    def custom_loss(y_true, y_pred):
#         print()
        margin = ((y_pred[:,:1] - 0.5) * 2)
        margin *= margin_size
        direction = tf.sign(margin)
        stop_loss = y_pred[:,1:2] * direction * -1 * stop_limit
        price_target = y_pred[:,2:3] * direction * 1 * price_limit
        
        idx = y_true[0].numpy()[1]
        df_name = y_true[0].numpy()[0]
        batch_size = y_pred.shape[0]
        closes = []
        spread = []
        for i in range(batch_size):
            closes.append(df[df_name].iloc[int(idx+i):int(idx+i+time_limit)]['close_1min'].tolist())
            spread.append(df[df_name].iloc[int(idx+i):int(idx+i+time_limit)]['spread'].tolist())
        closes = tf.convert_to_tensor(closes)
        spread = tf.convert_to_tensor(spread)
        spread *= direction
        
        close1 = closes[:,:1]
#         print(y_pred[0].numpy())
#         print(margin[0].numpy()[0], stop_loss[0].numpy()[0], price_target[0].numpy()[0])
        close2 = tf.zeros_like(closes[:,:1])
        for i in range(1, closes.shape[1]):
            diff = (closes[:,i:i+1] - closes[:,:1]) * direction
            cond = tf.logical_or(tf.less(diff, stop_loss), tf.greater(diff, price_target))
            close2 = tf.where(tf.logical_and(tf.equal(close2, 0), cond), closes[:,i:i+1], close2)
            
        close2 = tf.where(tf.equal(close2, 0), closes[:,-1:], close2)

        max_diff = tf.maximum(tf.reduce_max(closes, axis=1, keepdims=True) - close1, close1 - tf.reduce_min(closes, axis=1, keepdims=True))
        max_profit = 100 * (margin_size * (max_diff - spread)) /  close1
        
        diff = close2 - close1
        profit = 100 * (margin * (diff - spread)) /  close1
#         print(margin[0].numpy()[0], max_diff[0].numpy()[0], diff[0].numpy()[0], max_profit[0].numpy()[0], profit[0].numpy()[0])
        loss = tf.keras.losses.MSE(max_profit, profit)
#         print(margin[0].numpy()[0], close1[0].numpy()[0], close2[0].numpy()[0], spread[0].numpy()[0], loss[0].numpy(), profit[0].numpy()[0])
        return loss
    return custom_loss

In [107]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=custom_loss_wrapper(idx, stop_limit, price_limit, margin_size, time_limit),
    metrics=[custom_metric_wrapper(idx, stop_limit, price_limit, margin_size, time_limit)],
    run_eagerly=True
)

In [108]:
buffer = (batch_size+time_limit) + 1000
history = model.fit(inputs_train[:-buffer], np.array([[0, i] for i in range(len(train)-buffer)]), 
                    validation_data=(inputs_test[:-buffer], np.array([[1, i] for i in range(len(test)-buffer)])),
                    epochs=1000, shuffle=False,
                    batch_size=batch_size,
                   use_multiprocessing=True, verbose=1, validation_freq=10)

Epoch 1/1000
2267/2267 [==============================] - 522s 230ms/step - loss: 491.6399 - custom_metric: -0.4567
Epoch 2/1000
2267/2267 [==============================] - 523s 231ms/step - loss: 497.2239 - custom_metric: -0.5815
Epoch 3/1000
2267/2267 [==============================] - 522s 230ms/step - loss: 508.9987 - custom_metric: -0.5899
Epoch 4/1000
2267/2267 [==============================] - 521s 230ms/step - loss: 481.3517 - custom_metric: -0.5747
Epoch 5/1000
2267/2267 [==============================] - 524s 231ms/step - loss: 483.6128 - custom_metric: -0.5895
Epoch 6/1000
2267/2267 [==============================] - 524s 231ms/step - loss: 488.0869 - custom_metric: -0.5609
Epoch 7/1000
2267/2267 [==============================] - 523s 231ms/step - loss: 489.2521 - custom_metric: -0.5652
Epoch 8/1000
2267/2267 [==============================] - 523s 231ms/step - loss: 485.5792 - custom_metric: -0.5731
Epoch 9/1000
2267/2267 [==============================] - 522s 230ms/ste

KeyboardInterrupt: 

In [112]:
preds = model.predict(inputs_test)

In [ ]:
idx = 0
for p in preds:
    margin = ((y_pred[:,:1] - 0.5) * 2)
    margin *= margin_size
    direction = tf.sign(margin)
    stop_loss = y_pred[:,1:2] * direction * -1 * stop_limit
    price_target = y_pred[:,2:3] * direction * 1 * price_limit

    df_name = int(y_true[0].numpy()[0])
    batch_size = y_pred.shape[0]
    closes = []
    spread = []
    diff = (closes[:,1:2] - closes[:,:1]) * direction
    while idx < len(df['test']) and 
        closes.append(df[df_name].iloc[int(idx+i):int(idx+i+time_limit)]['close_1min'].tolist())
        spread.append(df[df_name].iloc[int(idx+i):int(idx+i+time_limit)]['spread'].tolist())
    closes = tf.convert_to_tensor(closes)
    spread = tf.convert_to_tensor(spread)
    spread *= direction

    close1 = closes[:,:1]
    close2 = tf.zeros_like(closes[:,:1])
    for i in range(1, closes.shape[1]):
        diff = (closes[:,i:i+1] - closes[:,:1]) * direction
        cond = tf.logical_or(tf.less(diff, stop_loss), tf.greater(diff, price_target))
        close2 = tf.where(tf.logical_and(tf.equal(close2, 0), cond), closes[:,i:i+1], close2)

    close2 = tf.where(tf.equal(close2, 0), closes[:,-1:], close2)

    diff = close2 - close1
    profit = 100 * (margin * (diff - spread)) /  close1

In [113]:
model.predict(inputs_test[:10])

array([[1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01]], dtype=float32)

In [114]:
preds

array([[1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       ...,
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01],
       [1.000000e+00, 8.003792e-20, 9.996779e-01]], dtype=float32)